In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
words =open('names.txt','r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
chars= sorted(list(set(''.join(words))))
stoi= {s:i+1 for i,s in enumerate(chars)}
stoi['.']= 0
itos = {i:s for s,i in stoi.items()}

In [4]:
block_size= 3
X,Y = [],[]
for w in words:
    #print(w)
    context = [0]* block_size
    for ch in w + '.':
        ix= stoi [ch]
        X.append(context)
        Y.append(ix)
        #print((''.join(itos[i] for i in context), '----->', itos[ix]))
        context = context[1:] + [ix]
X= torch.tensor(X)
Y= torch.tensor(Y)

In [5]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [70]:
#splitting the data into training, test, validation splits:
def build_dataset(words):
    block_size= 3
    X,Y = [],[]
    for w in words:
        #print(w)
        context = [0]* block_size
        for ch in w + '.':
            ix= stoi [ch]
            X.append(context)
            Y.append(ix)
            #print((''.join(itos[i] for i in context), '----->', itos[ix]))
            context = context[1:] + [ix]
    X= torch.tensor(X)
    Y= torch.tensor(Y)
    print (X.shape, Y.shape)
    return X , Y

import random
random.seed(42)
random.shuffle(words)
n1= int(0.8* len(words))
n2= int(0.9* len (words))
XTR , YTR = build_dataset(words[:n1])
XVAL , YVAL = build_dataset(words[n1:n2])
XTES , YTES = build_dataset(words[n2:])

torch.Size([182534, 3]) torch.Size([182534])
torch.Size([22873, 3]) torch.Size([22873])
torch.Size([22739, 3]) torch.Size([22739])


In [6]:
C= torch.randn(27,2)

In [7]:
C[5]

tensor([ 0.9219, -1.2711])

In [8]:
C[torch.tensor([5,6,7])]

tensor([[ 0.9219, -1.2711],
        [-1.2691, -0.2001],
        [-0.5611,  0.6829]])

In [9]:
emb = C[X]
emb.shape

torch.Size([228146, 3, 2])

In [10]:
torch.cat(torch.unbind(emb, 1),1).shape #less efficeint

torch.Size([228146, 6])

In [11]:
#emb.view(32,6)== torch.cat(torch.unbind(emb, 1),1)

In [12]:
W1=torch.randn (6,100)
B1=torch.randn(100)

In [13]:
h = torch.tanh(emb.view(-1,6) @ W1 + B1)

In [14]:
W2= torch.randn(100,27)
B2= torch.randn(27)
logits = h @ W2 + B2

In [15]:
logits.shape

torch.Size([228146, 27])

In [16]:
counts= logits.exp()

In [17]:
prob = counts/counts.sum(1, keepdims= True)

In [18]:
prob.shape

torch.Size([228146, 27])

In [19]:
#loss= -prob[torch.arange(32),Y].log().mean()
#loss

In [20]:
#_________________more organized_____________________

In [114]:
#non hard coded dynamic approach
nletter = 27
nemb = 10
nhidden = 300
block_size = 3
n_in = block_size* nemb
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((nletter, nemb), generator=g)
W1 = torch.randn((n_in, nhidden), generator=g)
b1 = torch.randn(nhidden, generator=g)
W2 = torch.randn(nhidden, nletter, generator=g)
b2 = torch.randn(nletter, generator=g)
parameters = [C, W1, b1, W2, b2]

#g = torch.Generator().manual_seed(2147483647) # for reproducibility
#C = torch.randn((27, 10), generator=g)
#W1 = torch.randn((30, 300), generator=g)
#b1 = torch.randn(300, generator=g)
#W2 = torch.randn((300, 27), generator=g)
#b2 = torch.randn(27, generator=g)
#parameters = [C, W1, b1, W2, b2]

In [115]:
sum(p.nelement() for p in parameters)

17697

In [116]:
for p in parameters:
    p.requires_grad = True

In [101]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10 ** lre

In [144]:
lri= []
lossi= []
for i in range (20000):
    ix = torch.randint(0, XTR.shape[0],(32,))
    emb = C[XTR[ix]]
    h = torch.tanh(emb.view(-1,n_in) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits,YTR[ix]) 
    #print(loss.item())
    for p in parameters:
        p.grad = None
    loss.backward()
    #lr=lrs[i]
    lr= 0.02
    for p in parameters:
        p.data += -lr * p.grad
    #lri.append(lr)
    #lossi.append(loss.item())
#print(loss.item())

In [88]:
#plt.plot(lri,lossi)

In [145]:
emb = C[XVAL]
h = torch.tanh(emb.view(-1,n_in) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits,YVAL) 
print(loss.item())

2.2113852500915527


In [146]:
emb = C[XTES]
h = torch.tanh(emb.view(-1,n_in) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits,YTES) 
print(loss.item())

2.220763921737671


In [147]:
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      emb = C[torch.tensor([context])] # (1,block_size,d)
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

carmahxati.
hari.
kimri.
reety.
salayane.
rahnel.
deliah.
jareei.
ner.
kia.
chaiir.
kaleigh.
ham.
joce.
quint.
shoilea.
jamii.
waver.
giearynix.
kaedeighan.
